In [1]:
from gurobipy import *

# 創建模型
m = Model("Robot and Cargo Path Planning")

# 定義網格大小和時間上限
grid_size, T, num_robots, num_cargos = 8, 20, 3, 4

# 變量
x = m.addVars(num_robots, grid_size, grid_size, T+1, vtype=GRB.BINARY, name="robot")  # 機器人的位置變量
c = m.addVars(num_cargos, grid_size, grid_size, T+1, vtype=GRB.BINARY, name="cargo")  # 貨物的位置變量
z = m.addVars(num_robots, num_cargos, T+1, vtype=GRB.BINARY, name="carrying")         # 機器人搬運貨物的變量
y = m.addVars(num_cargos, T+1, vtype=GRB.BINARY, name="finish")                       # 貨物到達終點的變量

# 權重參數
alpha = 0.0001  

# 目標函數，最小化所有貨物的到達時間總和和所有機器人的移動總次數
m.setObjective(
    quicksum(t * (y[c_idx, t] - y[c_idx, t-1]) for c_idx in range(num_cargos) for t in range(1, T+1)) + 
    alpha * quicksum(x[r, i, j, t] * (1 - x[r, i, j, t-1]) for r in range(num_robots) 
                     for i in range(grid_size) for j in range(grid_size) for t in range(1, T+1)),
    GRB.MINIMIZE
)

# 設定初始位置和目標位置
initial_positions = [(0, 1, 3), (1, 0, 2), (2, 1, 6)]  # 機器人的初始位置
cargo_init_pos = [(0, 0, 3), (1, 3, 3), (2, 2, 2), (3, 1, 2)]  # 貨物的初始位置
cargo_goal_pos = [(0, 1, 3), (1, 6, 7), (2, 4, 2), (3, 4, 1)]  # 貨物的目標位置

# 機器人的初始位置
for r, i, j in initial_positions:
    m.addConstr(x[r, i, j, 0] == 1)

# 貨物的初始位置
for c_idx, i, j in cargo_init_pos:
    m.addConstr(c[c_idx, i, j, 0] == 1)

# 貨物的目標位置
for c_idx, i, j in cargo_goal_pos:
    m.addConstr(y[c_idx, T] == 1)  # 確保在最後時刻 T 時，所有貨物都到達終點
    for t in range(1, T+1):
        m.addConstr(y[c_idx, t] <= c[c_idx, i, j, t])
        m.addConstr(y[c_idx, t] >= y[c_idx, t-1])  # 一旦到達終點，狀態保持到達

# 每個格子在每個時刻只能有一個機器人或貨物
for t in range(T+1):
    for i in range(grid_size):
        for j in range(grid_size):
            m.addConstr(quicksum(x[r, i, j, t] for r in range(num_robots)) <= 1)  # 機器人
            m.addConstr(quicksum(c[cargo, i, j, t] for cargo in range(num_cargos)) <= 1)  # 貨物

# 機器人和貨物在每個時刻只會在一個格子
for t in range(T+1):
    for r in range(num_robots):
        m.addConstr(quicksum(x[r, i, j, t] for i in range(grid_size) for j in range(grid_size)) == 1)
    for cargo in range(num_cargos):
        m.addConstr(quicksum(c[cargo, i, j, t] for i in range(grid_size) for j in range(grid_size)) == 1)

# 機器人和貨物的移動
for t in range(T):
    for r in range(num_robots):  #機器人只能在相鄰位置之間移動
        for i in range(grid_size):
            for j in range(grid_size):
                m.addConstr(x[r, i, j, t+1] <= x[r, i, j, t] + 
                            quicksum(x[r, i1, j1, t] 
                                     for i1, j1 in [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]
                                     if 0 <= i1 < grid_size and 0 <= j1 < grid_size))

    for cargo in range(num_cargos):   #貨物的移動需要由機器人來完成，並且只能在相鄰位置之間移動
        for i in range(grid_size):
            for j in range(grid_size):
                m.addConstr(c[cargo, i, j, t+1] <= c[cargo, i, j, t] + 
                            quicksum(z[r, cargo, t] * x[r, i, j, t+1] for r in range(num_robots)))

# 機器人搬運貨物的約束
for t in range(T):
    for r in range(num_robots):
        for cargo in range(num_cargos):
            m.addConstr(z[r, cargo, t] <= quicksum(x[r, i, j, t] * c[cargo, i, j, t] 
                                     for i in range(grid_size) for j in range(grid_size)))


# 機器人和貨物搬運數量的限制
for t in range(T+1):
    for r in range(num_robots):
        m.addConstr(quicksum(z[r, cargo, t] for cargo in range(num_cargos)) <= 1)  # 每個機器人同時只能搬運一個貨物
    for cargo in range(num_cargos):
        m.addConstr(quicksum(z[r, cargo, t] for r in range(num_robots)) <= 1)  # 每個貨物同時只能被一個機器人搬運

# 每個貨物在時間 T 之前至少到達終點一次
for cargo in range(num_cargos):
    goal_i, goal_j = cargo_goal_pos[cargo][1], cargo_goal_pos[cargo][2]
    m.addConstr(quicksum(c[cargo, goal_i, goal_j, t] for t in range(T+1)) >= 1)

# 求解模型
m.optimize()

# 輸出結果
if m.status == GRB.OPTIMAL:
    for t in range(T+1):
        for r in range(num_robots):
            robot_pos = [(i+1, j+1) for i in range(grid_size) for j in range(grid_size) if x[r, i, j, t].x > 0.5]
            print(f"t={t}: 機器人{r+1}在 {robot_pos}", end="")
            
            carrying = [cargo+1 for cargo in range(num_cargos) if z[r, cargo, t].x > 0.5]
            if carrying:
                print(f", 正在搬運貨物{carrying[0]}", end="")
            print()
        
        for cargo in range(num_cargos):
            cargo_pos = [(i+1, j+1) for i in range(grid_size) for j in range(grid_size) if c[cargo, i, j, t].x > 0.5]
            goal_i, goal_j = cargo_goal_pos[cargo][1] + 1, cargo_goal_pos[cargo][2] + 1  
            
            if cargo_pos == [(goal_i, goal_j)]:
                print(f"t={t}: 貨物{cargo+1}在 {cargo_pos}, 貨物已到達")
            else:
                print(f"t={t}: 貨物{cargo+1}在 {cargo_pos}")
        print()
else:
    print("無法找到解")


Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-10
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6997 rows, 9744 columns and 40855 nonzeros
Model fingerprint: 0xa698b56e
Model has 3840 quadratic objective terms
Model has 5360 quadratic constraints
Variable types: 0 continuous, 9744 integer (9744 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e-04, 2e+01]
  QObjective range [2e-04, 2e-04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1594 rows and 2458 columns
Presolve time: 0.49s
Presolved: 59542 rows, 32248 columns, 160296 nonzeros
Variable types: 0 continuous, 32248 inte